# Worm Communication

#### Config

In [1]:
will_connect = True

## Setup Interface

#### Imports

In [2]:
import serial
import ipywidgets
import threading
from ipywidgets import widgets
import time 
from IPython.display import display

#### Interface (variables)

In [3]:
tThresh = 10.0 # time threshold for holding direction (seconds)
direction_enum = [0,0] # 1 if is turning [left,right]

#### Interface - main

In [4]:
red_button = widgets.Button(
    description = "Shut Down",
    button_style='danger'
)    

send_button = widgets.Button(
    description = "Send Command",
    button_style='success'
)  

console_style = "style=\"width: 500px; color: steelblue; border-left: 6px solid darkslateblue; background-color: lightblue;\""
output = widgets.HTML(
    value="<dir "+console_style+"> empty </dir>",
    placeholder='Empty',
    description='',
    layout=ipywidgets.Layout(
        display='flex',
        width='100%'),
    disabled=True
)

command = widgets.Text(
    value='',
    placeholder='Enter command',
    description='',
    layout=ipywidgets.Layout(
        display='flex',
        width='80%'),
    disabled=False
)

#### Interface - Actuation

In [5]:
orientation = widgets.SelectionSlider(
    options=['backward', 'paused', 'forward'],
    value='paused',
    description='',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    layout=ipywidgets.Layout(display='flex',
                    width='100%'),
    readout=False
)

direction = widgets.SelectionSlider(
    options=['left', 'neutral' ,'right'],
    value='neutral',
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    layout=ipywidgets.Layout(display='flex',
                    width='100%'),
    readout=False
)

#### Interface - Data

In [6]:
leftMeter = widgets.FloatProgress(
    value=0,
    min=0,
    max=tThresh,
    step=0.1,
    description='Left Actuator Fatigue',
    bar_style='danger',
    orientation='vertical',
    layout=widgets.Layout(width='150px')
)

rightMeter = widgets.FloatProgress(
    value=0,
    min=0,
    max=tThresh,
    step=0.1,
    description='Right Actuator Fatigue',
    bar_style='danger',
    orientation='vertical',
    layout=widgets.Layout(width='150px')
)

#### Functions

In [7]:
def clicked():
    ser.write("X".encode())

def exeL():
    while (direction_enum[0] == 1 and leftMeter.value < leftMeter.max):
        time.sleep(0.1)
        leftMeter.value += 0.1
        if (leftMeter.value > leftMeter.max*0.7):
            leftMeter.bar_style='danger'
        else:
            leftMeter.bar_style='warning'
    direction_enum[0] = 0
    direction.value = "neutral"
    while (direction_enum[0] == 0 and leftMeter.value > leftMeter.min):
        time.sleep(0.1)
        leftMeter.value -= 0.2
        if (leftMeter.value < leftMeter.max*0.3):
            leftMeter.bar_style='success'
        else:
            leftMeter.bar_style='warning'

def exeR():
    while (direction_enum[1] == 1 and rightMeter.value < rightMeter.max):
        time.sleep(0.1)
        rightMeter.value += 0.1
        if (rightMeter.value > rightMeter.max*0.7):
            rightMeter.bar_style='danger'
        else:
            rightMeter.bar_style='warning'
    direction_enum[1] = 0
    direction.value = "neutral"
    while (direction_enum[1] == 0 and rightMeter.value > rightMeter.min):
        time.sleep(0.1)
        rightMeter.value -= 0.2
        if (rightMeter.value < rightMeter.max*0.3):
            rightMeter.bar_style='success'
        else:
            rightMeter.bar_style='warning'

def on_command_change(change):
    # actuation commands
    if (command.value != '' and len(command.value) == 1):
        # forward or backward
        if (command.value == "w"):
            if (orientation.value == "paused"):
                orientation.value = "forward"
            elif (orientation.value == "backward"):
                orientation.value = "paused"
        elif (command.value == "s"):
            if (orientation.value == "paused"):
                orientation.value = "backward"
            elif (orientation.value == "forward"):
                orientation.value = "paused"
        # left or right
        elif (command.value == "a"):
            if (direction.value == "neutral"):
                direction.value = "left"
                # count running time
                direction_enum[0] = 1
                lT = threading.Thread(target=exeL, args=())
                lT.start()
            elif (direction.value == "right"):
                direction.value = "neutral"
                direction_enum[1] = 0
        elif (command.value == "d"):
            if (direction.value == "neutral"):
                direction.value = "right"
                # count running time
                direction_enum[1] = 1
                rT = threading.Thread(target=exeR, args=())
                rT.start()
            elif (direction.value == "left"):
                direction.value = "neutral"
                direction_enum[0] = 0
        elif (command.value[0] == ":"):
            return
        command.value = ''
    # other commands have to start with ':'
    elif (command.value != '' and command.value[0] != ":"):
        command.value = ''
    

def on_orientation_change(change):
    #if (command.value != ''):
    if (orientation.value == "forward"):
        ser.write("f".encode())
    elif (orientation.value == "backward"):
        ser.write("b".encode())
    elif (orientation.value == "paused"):
        ser.write("p".encode())
    
def on_direction_change(change):
    #if (command.value != ''):
    if (direction.value == "left"):
        ser.write("l".encode())
    elif (direction.value == "right"):
        ser.write("r".encode())
    elif (direction.value == "neutral"):
        ser.write("m".encode())

## Setup Communication

#### Serial communication 

In [8]:
baudrate = 9600 # default, might need to adjust
serial_port = '/dev/tty.HC-06-DevB' # connect to the new hc-06 board
if will_connect:
    ser = serial.Serial(serial_port, baudrate, timeout=1) # open connection

#### Receiving from worm

In [9]:
def handle_data(data, text_list):
    # print(data)
    text_list = text_list[1:]+[time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())+"|||"+data]
    display_text = ""
    for x in text_list:
        display_text += x + "<br>"
    output.value = "<dir "+console_style+">" + display_text + "</dir>"
    return text_list

def read_from_port(ser):
    text_list = ["-\n","-\n","-\n","-\n","-\n","-\n","-\n","-\n","-\n","-\n"]
    while True:
        reading = ser.readline().decode()
        text_list = handle_data(reading, text_list)

### Start receiving thread; Display interface

In [10]:
if will_connect:
    thread = threading.Thread(target=read_from_port, args=(ser,))
    thread.start()

In [11]:
B1 = widgets.HBox([leftMeter, orientation, rightMeter])
B2 = widgets.HBox([red_button, command, send_button])
B3 = widgets.VBox([B1, direction, B2, output])
B3.layout=widgets.Layout(width='50%', margin='0 25% 0 25%')
B4 = widgets.VBox([B2, output])

display(B3)

red_button.on_click(clicked)
command.observe(on_command_change, names='value')
if will_connect:
    orientation.observe(on_orientation_change, names='value')
    direction.observe(on_direction_change, names='value')